In [50]:
import time
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix

In [51]:
# transform  = transforms.ToTensor()
trans_train = transforms.Compose([transforms.Lambda(lambda image: image.convert('RGB')),
                                  transforms.ColorJitter(brightness=0.05, contrast=0.8, saturation=0.02, hue=0.02),
                                  transforms.Resize((64,64)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (1.0,))])

trans = transforms.Compose([transforms.Lambda(lambda image: image.convert('RGB')),
                            transforms.ColorJitter(brightness=0.05, contrast=0.8, saturation=0.02, hue=0.02),
                            transforms.Resize((64,64)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (1.0,))])


train_data = dset.MNIST(root='../Data', train=True, download=True, transform=trans_train)
test_data = dset.MNIST(root='../Data', train=False, download=False, transform=trans)

In [52]:
print(train_data)
image,label=train_data[10]
print(image.shape)
print(label)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../Data
    Split: Train
    StandardTransform
Transform: Compose(
               Lambda()
               ColorJitter(brightness=(0.95, 1.05), contrast=(0.19999999999999996, 1.8), saturation=(0.98, 1.02), hue=(-0.02, 0.02))
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5,), std=(1.0,))
           )
torch.Size([3, 64, 64])
3


In [53]:
#loader data untuk training dan testing
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)

In [54]:
class SimpleNet(nn.Module):
    def __init__(self, num_classes):
        super(SimpleNet, self).__init__()
        #bagian 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.max_poo1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU(inplace=True)
        self.max_poo2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        # bagian 2
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU(inplace=True)
        self.max_poo3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.ReLU(inplace=True)
        self.max_poo4 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        #bagian classifier
        self.fc1 = nn.Linear(256, 4096) 
        # self.relu5 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(p=0.1)
        self.fc4 = nn.Linear(4096, num_classes)
        self.classifier = nn.Softmax(dim=1)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.max_poo1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.max_poo2(out)
        
        out = self.conv_layer3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.max_poo3(out)
        out = self.conv_layer4(out)
        out = self.bn4(out)
        out = self.relu4(out)
        out = self.max_poo4(out)
                
        out = out.view(out.size(0), -1) # mengubah dimensi tensor
        
        out = self.fc1(out)
        # out = self.relu5(out)
        out = self.dropout1(out)
        out = self.fc4(out)
        out = self.classifier(out)
        return out

In [55]:
model = SimpleNet(num_classes = 10)
model

SimpleNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (max_poo1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (max_poo2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (max_poo3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU(inpla

In [56]:
def train(epoch):
    model.train()
    start_time = time.time()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # if torch.cpu.is_available():
        #     data, target = data.cpu(), target.cpu()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        
        # Menghitung jumlah prediksi yang benar
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('\rEpoch: {} {:.0f}%\t     Loss: {:.6f}'.format(
                epoch,
                100. * batch_idx / len(train_loader), loss.item()), end='')
    end_time = time.time()
    print("\nLama waktu Training pada epoch {}: {} Menit".format(epoch, ((end_time - start_time)/60)))
    
    # Menghitung dan mencetak akurasi pelatihan
    train_accuracy = 100. * correct / len(train_loader.dataset)
    train_akurasi.append(train_accuracy)
    print('Akurasi pada Training pada epoch {}: {:.2f}%'.format(epoch, train_accuracy))

In [57]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        # if torch.cpu.is_available():
        #     data, target = data.cpu(), target.cpu()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    
    
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    acc=100. * float(correct.to(torch.device('cpu')).numpy())
    print('\nHasil Testing: Rata-Rata loss: {:.4f}, Akurasi: {:.4f}%\n'.format(
        test_loss, acc / len(test_loader.dataset)))
    
    test_accuracy.append(acc / len(test_loader.dataset))

In [58]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

train_losses = []
test_losses =[]
test_accuracy = []
train_akurasi = []

for epoch in range(1, 2):
    train(epoch)
    test()

Epoch: 1 96%	     Loss: 1.591911
Lama waktu Training pada epoch 1: 4.317583632469177 Menit
Akurasi pada Training pada epoch 1: 81.66%

Hasil Testing: Rata-Rata loss: 1.5222, Akurasi: 96.6500%



In [61]:
# membuat prediksi dan ground truth labels
y_pred = []
y_true = []
for data, target in test_loader:
    data, target = Variable(data), Variable(target)
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1].cpu().numpy().squeeze()
    y_pred.extend(pred)
    y_true.extend(target.data.cpu().numpy())

# membuat confusion matrix dan classification report
conf_mat = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)

print('Confusion Matrix:')
print(conf_mat)

Confusion Matrix:
[[ 963    0    7    0    0    2    7    1    0    0]
 [   0 1125    5    1    0    1    2    1    0    0]
 [   6    2  987    1   10    0    7   18    1    0]
 [   1    1   18  971    0    4    1   10    1    3]
 [   1    3    1    0  964    0    6    0    0    7]
 [   5    5    1   12    1  858    6    1    3    0]
 [   9    5    0    0    3    3  938    0    0    0]
 [   0    3   29    1    0    1    0  990    0    4]
 [   8    3   17    3    5    6    6    8  912    6]
 [   9    8    5    3    8    5    0   10    2  959]]


In [60]:
print('Classification Report:')
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       980
           1       0.98      0.99      0.98      1135
           2       0.92      0.95      0.94      1032
           3       0.98      0.96      0.97      1010
           4       0.97      0.98      0.98       982
           5       0.98      0.97      0.97       892
           6       0.97      0.98      0.98       958
           7       0.95      0.96      0.96      1028
           8       0.99      0.94      0.96       974
           9       0.98      0.95      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

